Пермяков Дмитрий ЛР4 КИС

## Задание 1.

Вычислительный центр фирмы состоит из $m$ главных серверов коллективного пользования. Число работающих в центре программистов в любой момент времени равно $k$. Каждый программист готовит свою программу и через терминал передает ее на сервер для выполнения, куда она сразу попадает. Время подготовки программ имеет экспоненциальное распределение со средним значением $t_{1}$ мин. Время выполнения программы на любом из серверов имеет экспоненциальное распределение со средним значением $t_{2}$ мин. Каждый программист ожидает ответа от сервера, прежде, чем начнет писать следующую программу.

Найти (теоретически и экспериментально):
 - вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал;
 - среднее время до получения пользователем результатов реализации;
 - среднее количество программ, ожидающих выполнения на сервере.

Требуется выполнить расчет при заданном по варианту значении $m$, а также при $m=1$.

In [ ]:
Variant<-13
set.seed(Variant)
k<-sample(c(10:25),1)
m<-sample(c(3:6),1)
t1<-sample(c(14:20),1)
t2<-sample(c(2:5),1)
View(data.frame(k,m,t1,t2))

k,m,t1,t2
<int>,<int>,<int>,<int>
17,5,18,3


# Для m по варианту

## Теоретическая часть

### Формула Эрланга

$$
P_{0} = \left( \sum_{i = 0}^{m} \frac{\frac{k!}{(k - i)!}}{i!}y^{i} + \sum_{j = m + 1}^{k} \frac{\frac{k!}{(k - j)!}}{m!m^{j - m}}y^{j} \right)^{-1}
$$

### Длина очереди

$$
L_{queue} = \sum_{r = 1}^{k - m} r P_{m + r} = \sum_{r = 1}^{k - m} r \frac{\frac{k!}{(k - m - r)!}}{m!m^{r}} y^{m + r} P_{0}
$$

### Вероятность необработки заявки

$$
P_{busy} = \sum_{r = 0}^{k - m} P_{m + r} = \sum_{r = 0}^{k - m} \frac{\frac{k!}{(k - m - r)!}}{m!m^{r}} y^{m + r} P_{0}
$$

### Среднее число заявок в системе

$$
L_{sys} = \sum_{i = 0}^{m} i P_{i} + \sum_{r = 1}^{k - m} (r + m) P_{m + r} = \sum_{i = 0}^{m} i \frac{\frac{k!}{(k - i)!}}{i!} y^{i} P_{0} + \sum_{r = 1}^{k - m} r \frac{\frac{k!}{(k - m - r)!}}{m!m^{r}} y^{m + r} P_{0}
$$

### Средняя интенсивность суммарного входного потока

$$
\Lambda = (k - L_{sys}) * \lambda
$$

### Время нахождения заявок в системе

$$
T_{sys} = \frac{L_{sys}}{\Lambda}
$$

### Вычислим для m по варианту:

In [ ]:
lambda <- 1 / t1
mu <- 1 / t2
y <- lambda / mu
P0 <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  P0 <- P0 + sum
}

for(j in (m + 1):k) {
  sum <-(factorial(k)/factorial(k - j)) / (factorial(m) * m^(j - m)) * y^j
  P0 <- P0 + sum
}

P0 <- 1 / P0
P0

[1] 0.07204052

In [ ]:
L_queue <- 0

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  L_queue <- L_queue + r*sum*P0
}

L_queue

[1] 0.05304379

In [ ]:
Pbusy <- 0

for (r in 0:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Pbusy <- Pbusy + sum*P0
}

Pbusy

[1] 0.0926007

In [ ]:
Lsys <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  Lsys <- Lsys + i*sum*P0
}

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Lsys <- Lsys + (r+m)*sum*P0
}

Lsys

[1] 2.474038

In [ ]:
ML = (k - Lsys) * lambda
ML

[1] 0.8069979

In [ ]:
Tsys = Lsys/ML
Tsys

[1] 3.06573

In [ ]:
bar_string <- "--------------------------------------------------------------"
View(bar_string)
View("среднее время до получения пользователем результатов реализации")
View(Tsys)
View(bar_string)
View("среднее количество программ, ожидающих выполнения на сервере.")
View(L_queue)
View(bar_string)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
View(Pbusy)

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 3.06573

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 0.05304379

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.0926007

 ## Экспериментальная часть

In [ ]:
Tend <- 10000
tsys <- 0

# Таблица с информацией о программистах и их программах
progs <- data.frame(
  end = tsys + rexp(1, 1 / t1),
  start = rep(0, k),
  prog_id = rep(0, k)
)
colnames(progs) <- c("end", "start", "deploy_prog_id")

servers <- data.frame(
  end = rep(Tend + 1, m),
  start = rep(0, m),
  prog_id = rep(0, m)
)
colnames(servers) <- c("end", "start", "prog_id")

counter_deploy <- 0 # количество отправленных в систему программ
counter_ready <- 0  # количество завершённых программ
system_count <- 1   # счётчик состояния системы
last_switch <- 0    # последнее время изменения состояния системы

Pn_vec <- c(0,0)    # 1 - количество программ, которые были выполнены сразу, 2 - количество программ, которые попали в очередь
L_queue_vec <- c(0) # массив для статистики по количеству программ в очереди

Tget_vec <- data.frame() # Время завершения работы программы
Tget_vec <- rbind(Tget_vec, c(0,0))
colnames(Tget_vec) <- c("time", "ready_flag")

queue <- c()           # очередь ожидающих выполнения программ
ready_time <- Tend + 1 # ближайшее время, когда сервер будет свободен

# Проверка, свободны ли серваки
if_free_servers <- function() {
  for (i in 1:m) {
    if (servers$prog_id[i] == 0) {
      return (1)
    }
  }
  return (0)
}


while (tsys < Tend) {
  l_och_index <- length(queue) + 1
  # Если вектор по текущему индексу ещё не сущ, создаём
  if (is.na(L_queue_vec[l_och_index])) {
    L_queue_vec[l_och_index] <- 0
  }
  # Учитывается, сколько времени длилось состояние с определённой длиной очереди
  L_queue_vec[l_och_index] <- L_queue_vec[l_och_index] + tsys - last_switch
  last_switch <- tsys

  # Обработка программ, которые завершили подготовку
  for (i in 1:k) {
    # Если у программиста закончилось время подготовки программы
    if (tsys == progs$end[i]) {
      if (counter_deploy - counter_ready < m) {
        # Если есть свободные сервера
        Pn_vec[1] <- Pn_vec[1] + 1
      } else {
         # Если сервера заняты
        Pn_vec[2] <- Pn_vec[2] + 1
      }

      counter_deploy <- counter_deploy + 1
      progs$deploy_prog_id[i] <- counter_deploy

      Tget_vec <- rbind(Tget_vec, c(0,0))
      Tget_vec$time[counter_deploy] <- progs$end[i] # обновляется время ожидания программы
      Tget_vec$ready_flag[counter_deploy] <- 0      # флаг готовности программы 0 - программа ещё не завершена

      progs$end[i] <- Tend + 1 # чтобы она не была выполнена снова

      queue <- append(queue, counter_deploy)
    }
  }

  # Обработка завершённых программ на серверах
  for (i in 1:m) {
    # Завершилась ли прога на серваке?
    if (tsys == servers$end[i]) {
      counter_ready <- counter_ready + 1
      Tget_vec$time[counter_ready] <- tsys - Tget_vec$time[counter_ready] # Время ожидания программы. Т.е now минус временя, когда она была добавлена в систему
      Tget_vec$ready_flag[counter_ready] <- 1

      for (j in 1:k) {
        # Ищем ту самую прогу по её id
        if (progs$deploy_prog_id[j] == servers$prog_id[i]) {
          progs$start[j] = tsys
          progs$end[j] = tsys + rexp(1, 1 / t1)
        }
      }

      servers$prog_id[i] <- 0
      servers$end[i] <- Tend + 1
    }
  }

  # Запуск программы на сервере
  if (length(queue)) {
    if (if_free_servers()) {
      # В случайном порядке найдём свободный сервер
      for (i in sample(1:m)) {
        if(servers$prog_id[i] == 0){
          servers$start[i] <- tsys
          servers$end[i] <- tsys + rexp(1, 1 / t2)
          servers$prog_id[i] <- queue[1]
          queue <- queue[-1] # Удаляем первую прогу из очереди
          break
        }
      }
    }
  }

  tsys <- min(progs$end[1:k], servers$end[1:m])
}

In [ ]:
# Отфильтровываем завершенные программы и выводим среднее время
Tget_vec <- Tget_vec[Tget_vec$ready_flag == 1,]
mean_time <- mean(Tget_vec$time)
cat("Среднее время до получения пользователем результатов реализации:", mean_time, "\n")

# Рассчитываем среднее количество программ в очереди
Loch_vec_P <- L_queue_vec / sum(L_queue_vec)
Loch <- sum((1:length(L_queue_vec) - 1) * Loch_vec_P)
cat("Среднее количество программ, ожидающих выполнения на сервере:", Loch, "\n")

# Вероятность того, что программа не будет выполнена сразу
Pn <- Pn_vec[2] / sum(Pn_vec)
cat("Вероятность того, что программа не будет выполнена сразу, как только она поступила на терминал:", Pn, "\n")

Среднее время до получения пользователем результатов реализации: 3.089704 
Среднее количество программ, ожидающих выполнения на сервере: 0.06176837 
Вероятность того, что программа не будет выполнена сразу, как только она поступила на терминал: 0.07505573 


# Для m = 1

 ## Экспериментальная часть

In [ ]:
m=1
Tend <- 10000
tsys <- 0

# Таблица с информацией о программистах и их программах
progs <- data.frame(
  end = tsys + rexp(1, 1 / t1),
  start = rep(0, k),
  prog_id = rep(0, k)
)
colnames(progs) <- c("end", "start", "deploy_prog_id")

servers <- data.frame(
  end = rep(Tend + 1, m),
  start = rep(0, m),
  prog_id = rep(0, m)
)
colnames(servers) <- c("end", "start", "prog_id")

counter_deploy <- 0 # количество отправленных в систему программ
counter_ready <- 0  # количество завершённых программ
system_count <- 1   # счётчик состояния системы
last_switch <- 0    # последнее время изменения состояния системы

Pn_vec <- c(0,0)    # 1 - количество программ, которые были выполнены сразу, 2 - количество программ, которые попали в очередь
L_queue_vec <- c(0) # массив для статистики по количеству программ в очереди

Tget_vec <- data.frame() # Время завершения работы программы
Tget_vec <- rbind(Tget_vec, c(0,0))

colnames(Tget_vec) <- c("time", "ready_flag")

queue <- c()           # очередь ожидающих выполнения программ
ready_time <- Tend + 1 # ближайшее время, когда сервер будет свободен

# Проверка, свободны ли серваки
if_free_servers <- function() {
  for (i in 1:m) {
    if (servers$prog_id[i] == 0) {
      return (1)
    }
  }
  return (0)
}


while (tsys < Tend) {
  l_och_index <- length(queue) + 1
  # Если вектор по текущему индексу ещё не сущ, создаём
  if (is.na(L_queue_vec[l_och_index])) {
    L_queue_vec[l_och_index] <- 0
  }
  # Учитывается, сколько времени длилось состояние с определённой длиной очереди
  L_queue_vec[l_och_index] <- L_queue_vec[l_och_index] + tsys - last_switch
  last_switch <- tsys

  # Обработка программ, которые завершили подготовку
  for (i in 1:k) {
    # Если у программиста закончилось время подготовки программы
    if (tsys == progs$end[i]) {
      if (counter_deploy - counter_ready < m) {
        # Если есть свободные сервера
        Pn_vec[1] <- Pn_vec[1] + 1
      } else {
         # Если сервера заняты
        Pn_vec[2] <- Pn_vec[2] + 1
      }

      counter_deploy <- counter_deploy + 1
      progs$deploy_prog_id[i] <- counter_deploy

      Tget_vec <- rbind(Tget_vec, c(0,0))
      Tget_vec$time[counter_deploy] <- progs$end[i] # обновляется время ожидания программы
      Tget_vec$ready_flag[counter_deploy] <- 0 # флаг готовности программы 0, что означает, что программа ещё не завершена

      progs$end[i] <- Tend + 1 # чтобы она не была выполнена снова

      queue <- append(queue, counter_deploy)
    }
  }

  # Обработка завершённых программ на серверах
  for (i in 1:m) {
    # Завершилась ли прога на серваке?
    if (tsys == servers$end[i]) {
      counter_ready <- counter_ready + 1
      Tget_vec$time[counter_ready] <- tsys - Tget_vec$time[counter_ready] # Время ожидания программы. Т.е now минус временя, когда она была добавлена в систему
      Tget_vec$ready_flag[counter_ready] <- 1

      for (j in 1:k) {
        # Ищем ту самую прогу по её id
        if (progs$deploy_prog_id[j] == servers$prog_id[i]) {
          progs$start[j] = tsys
          progs$end[j] = tsys + rexp(1, 1 / t1)
        }
      }

      servers$prog_id[i] <- 0
      servers$end[i] <- Tend + 1
    }
  }

  # Запуск программы на сервере
  if (length(queue)) {
    if (if_free_servers()) {
      # В случайном порядке найдём свободный сервер
      for (i in sample(1:m)) {
        if(servers$prog_id[i] == 0){
          servers$start[i] <- tsys
          servers$end[i] <- tsys + rexp(1, 1 / t2)
          servers$prog_id[i] <- queue[1]
          queue <- queue[-1] # Удаляем первую прогу из очереди
          break
        }
      }
    }
  }

  tsys <- min(progs$end[1:k], servers$end[1:m])
}

In [ ]:
Tget_vec <- Tget_vec[Tget_vec$ready_flag == 1,]
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
mean(Tget_vec$time)

Loch_vec_P <- Loch_vec / sum(Loch_vec)
Loch <- 0
for (i in 1:length(Loch_vec)) {
  Loch <- Loch + (i - 1) * Loch_vec_P[i]
}

View(paste)
View("среднее количество программ, ожидающих выполнения на сервере.")
Loch

Pn <- Pn_vec[2]/sum(Pn_vec)

View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
Pn

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 32.54431

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 9.995488

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.9994085

## Теоретическая

### Составим формулу Эрланга

$$
P_{0} = \left( 1 + \frac{\frac{k!}{(k - 1)!}}{1!} y + \frac{\frac{k!}{(k - 2)!}}{2!} y^{2} + \frac{\frac{k!}{(k - 3)!}}{3!} y^{3} + \dots + \frac{\frac{k!}{(k - m)!}}{m!} y^{m} + \frac{\frac{k!}{(k - m - 1)!}}{m! m} y^{m + 1} + \frac{\frac{k!}{(k - m - 2)!}}{m! m^{2}} y^{m + 2} + \dots + \frac{\frac{k!}{(k - (k - 1))!}}{m! m^{k - m - 1}} y^{k - 1} + \frac{\frac{k!}{(k - k)!}}{m! m^{k - m}} y^{k} \right)^{-1}
$$

### Свернём формулу

$$
P_{0} = \left( \sum_{i = 0}^{m} \frac{\frac{k!}{(k - i)!}}{i!} y^{i} + \sum_{j = m + 1}^{k} \frac{\frac{k!}{(k - j)!}}{m! m^{j - m}} y^{j} \right)^{-1}
$$

### Длина очереди

$$
L_{och} = \sum_{r = 1}^{k - m} r P_{m + r} = \sum_{r = 1}^{k - m} r \frac{\frac{k!}{(k - m - r)!}}{m! m^{r}} y^{m + r} P_{0}
$$

### Вероятность необработки заявки

$$
P_{busy} = \sum_{r = 0}^{k - m} P_{m + r} = \sum_{r = 0}^{k - m} \frac{\frac{k!}{(k - m - r)!}}{m! m^{r}} y^{m + r} P_{0}
$$

### Среднее число заявок в системе

$$
L_{sys} = \sum_{i = 0}^{m} i P_{i} + \sum_{r = 1}^{k - m} (r + m) P_{m + r} = \sum_{i = 0}^{m} i \frac{\frac{k!}{(k - i)!}}{i!} y^{i} P_{0} + \sum_{r = 1}^{k - m} r \frac{\frac{k!}{(k - m - r)!}}{m! m^{r}} y^{m + r} P_{0}
$$

### Средняя интенсивность суммарного входного потока

$$
\Lambda = (k - L_{sys}) \times \lambda
$$

### Время нахождения заявок в системе

$$
T_{sys} = \frac{L_{sys}}{\Lambda}
$$

### Вычислим:

In [ ]:
lambda <- 1 / t1
mu <- 1 / t2
y <- lambda / mu
P0 <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  P0 <- P0 + sum
}

for(j in (m + 1):k) {
  sum <-(factorial(k)/factorial(k - j)) / (factorial(m) * m^(j - m)) * y^j
  P0 <- P0 + sum
}

P0 <- 1 / P0
P0

[1] 0.000117967

In [ ]:
L_queue <- 0

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  L_queue <- L_queue + r*sum*P0
}

L_queue

[1] 10.00083

In [ ]:
Pbusy <- 0

for (r in 0:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Pbusy <- Pbusy + sum*P0
}

Pbusy

[1] 0.999882

In [ ]:
Lsys <- 0

for(i in 0:m) {
  sum <-(factorial(k)/factorial(k - i))/factorial(i) * y^i
  Lsys <- Lsys + i*sum*P0
}

for (r in 1:(k - m)) {
  sum = (factorial(k)/factorial(k-m-r))/(factorial(m)*m^r)*y^(m+r)

  Lsys <- Lsys + (r+m)*sum*P0
}

Lsys

[1] 11.00071

In [ ]:
ML = (k - Lsys) * lambda
ML

[1] 0.333294

In [ ]:
Tsys = Lsys/ML
Tsys

[1] 33.00602

In [ ]:
paste <- "--------------------------------------------------------------"
View(paste)
View("среднее время до получения пользователем результатов реализации")
View(Tsys)
View(paste)
View("среднее количество программ, ожидающих выполнения на сервере.")
View(L_queue)
View(paste)
View('вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал')
View(Pbusy)

[1] "--------------------------------------------------------------"

[1] "среднее время до получения пользователем результатов реализации"

[1] 33.00602

[1] "--------------------------------------------------------------"

[1] "среднее количество программ, ожидающих выполнения на сервере."

[1] 10.00083

[1] "--------------------------------------------------------------"

[1] "вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал"

[1] 0.999882